# Trabalho Final de Mecânica dos Sólidos: 
## PROJETO DE DUTOS PARA SISTEMA DE CONDICIONAMENTO DE AR

**Alunos:**

João Gabriel Schunk 

João Henrique Lima de Vasconcelos

João Vitor Bordin

---

## Introdução

A resolução do problema será seccionada em uma série de submódulos como mostra o diagrama abaixo:

<center><img src="./img/fluxograma.png" alt="ias" width="400"/></center>


### Perdas de Carga
Começa-se pela resolução das perdas de carga em cada uma das linhas e em A. A perda de carga na linha 2-3A é igual a perda de carga na linha 2-3B assim, a perda de carga em B não é calculada.

As perdas de  

### Trabalho da Bomba
Com as perdas de carga, calcula-se o trabalho da bomba 

###

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import inputs as i
import aux as aux
from scipy.optimize import minimize
from itertools import *
import pandas as pd

In [3]:
def cost_function(D_array, return_cost_dict=False):
    h_12, v_12 = aux.head_loss_12(i.Q, D_array[0], i.L12, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
    
    h_23A, v_23A = aux.head_loss_23A(i.Q, D_array[1], i.L23A, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
    h_A, v_A = aux.head_loss_A(i.Q, i.dA, i.LA, i.Leqcurv, i.ρ, i.µ, i.e_tref, return_velocity=True)
    
    h_23B, v_23B = aux.head_loss_23B(i.Q, D_array[2], i.L23B, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
    h_B, v_B = aux.head_loss_B(i.Q, i.dB, i.LB, i.Leqcurv, i.ρ, i.µ, i.e_tref, return_velocity=True)

    h_31, v_31 = aux.head_loss_31(i.Q, D_array[3], i.L31, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
    
    W_bomba = aux.calc_W_bomba(h_12, h_23A, h_A, h_31, i.ρ, i.Q)

    cost = aux.calc_C_total(D_array, i.L12, i.L23A, i.L23B, i.L31, i.b, i.F, i.C2, i.t, W_bomba, i.n, i.a)
    
    cost_dict={"cost": cost,
               "v_12": v_12,
               "v_23A": v_23A,
               "v_A": v_A,
               "v_23B": v_23B,
               "v_B": v_B,
                "v_23A": v_31,
               "W_bomba": W_bomba} 

    if return_cost_dict:
        return cost_dict
    
    else:
        return cost


# Diametros comerciais
A norma NBR 5590 (ASTM A-53) / A106 A / API 5L B dita quais os possíveis diâmetros comerciais a tabela a seguir referencia todos os diâmetros: https://acotubo.com.br/tabelas-site/tubos-de-aco/tubos-de-conducao-com-e-sem-costura-nbr-5590-astm-53-a106-api-5l-b.html 

In [16]:
comercial_diameters = pd.read_excel("diametros_comerciais.xlsx", skiprows = [1,2], usecols=range(1))
comercial_diameters.dropna(subset=["Diâmetro"], inplace=True)

comercial_diameters = comercial_diameters.to_numpy()
#comercial_diameters.flatten()
possible_diameters = []
for diameter in comercial_diameters.flatten():
    diameter = eval(diameter.replace("”", "").replace(".", "+"))
    possible_diameters.append(diameter)

df0 = pd.DataFrame()
df0["Diametros Comerciais"] = possible_diameters

display(df0)

,Diametros Comerciais
0,0.250
1,0.375
2,0.500
3,0.750
4,1.000
5,1.250
6,1.500
7,2.000
8,2.500
9,3.000


## Valores iniciais dos diâmetros

In [4]:
D_guess = np.zeros(4)
D_guess[0] = 1.0 # [in]
D_guess[1] = 1.0 # [in]
D_guess[2] = 1.0 # [in]
D_guess[3] = 1.0 # [in]
D_guess

array([1., 1., 1., 1.])

In [13]:
sol = minimize(cost_function, D_guess*2.54e-2, bounds=[(0.250*2.54e-2, 24.0*2.54e-2), (0.250*2.54e-2, 24.0*2.54e-2), (0.250*2.54e-2, 24.0*2.54e-2), (0.250*2.54e-2, 24.0*2.54e-2)])

print("Custo total = R$", sol["fun"])

Custo total = R$ 1405708.3554023164


In [14]:
print(sol)

      fun: 1405708.3554023164
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 9.47853550e+01, -2.43540853e+01,  9.74054914e+05,  8.90577212e+01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 11
     njev: 21
   status: 0
  success: True
        x: array([0.04281124, 0.0367772 , 0.00635   , 0.04767688])


In [15]:
ideal_diameters = sol["x"]/2.54e-2
ideal_diameters

array([1.68548205, 1.44792138, 0.25      , 1.87704237])

In [17]:
all_configurations = [item for item in product(possible_diameters, repeat=len(D_guess))]

all_configurations = np.array([list(elem) for elem in all_configurations])

In [18]:
display(pd.DataFrame(all_configurations,
                columns=["Diametro da Linha 1-2 [in]",
                         "Diametro da Linha 2-3A [in]",
                         "Diametro da Linha 2-3B [in]",
                         "Diametro da Linha 3-1 [in]"]))

,Diametro da Linha 1-2 [in],Diametro da Linha 2-3A [in],Diametro da Linha 2-3B [in],Diametro da Linha 3-1 [in]
0,0.25,0.25,0.25,0.250
1,0.25,0.25,0.25,0.375
2,0.25,0.25,0.25,0.500
3,0.25,0.25,0.25,0.750
4,0.25,0.25,0.25,1.000
...,...,...,...,...
279836,24.00,24.00,24.00,16.000
279837,24.00,24.00,24.00,18.000
279838,24.00,24.00,24.00,20.000
279839,24.00,24.00,24.00,22.000


In [19]:
costs=[]
for D_array in all_configurations:
    cost = cost_function(2.54e-2*D_array)
    costs.append(cost)
costs = np.array(costs)

In [20]:
df = pd.DataFrame(all_configurations,
                columns=["Diametro da Linha 1-2 [in]",
                         "Diametro da Linha 2-3A [in]",
                         "Diametro da Linha 2-3B [in]",
                         "Diametro da Linha 3-1 [in]"])
df["Custo total da linha [R$]"] = costs
df.index.name = "Iteração"

result = df.sort_values("Custo total da linha [R$]", ascending=True)

display(result)

,Diametro da Linha 1-2 [in],Diametro da Linha 2-3A [in],Diametro da Linha 2-3B [in],Diametro da Linha 3-1 [in],Custo total da linha [R$]
Iteração,,,,,
76183,1.50,1.50,0.250,2.00,1.410909e+06
88350,2.00,1.50,0.250,2.00,1.412848e+06
76206,1.50,1.50,0.375,2.00,1.414002e+06
88373,2.00,1.50,0.375,2.00,1.415941e+06
76229,1.50,1.50,0.500,2.00,1.417094e+06
...,...,...,...,...,...
414,0.25,0.25,16.000,0.25,7.530679e+08
437,0.25,0.25,18.000,0.25,7.531174e+08
460,0.25,0.25,20.000,0.25,7.531669e+08


In [21]:
comercial_diameters = result.iloc[0][0:-1].to_numpy()*2.54e-2

comercial_diameters/2.54e-2

array([1.5 , 1.5 , 0.25, 2.  ])

In [22]:
ideal_diameters

array([1.68548205, 1.44792138, 0.25      , 1.87704237])

In [23]:
cost_function(ideal_diameters, return_cost_dict=True)

{'cost': 11649210.307905326,
 'h_12': 1.1946873942854095e-05,
 'v_12': 0.001244971346017479,
 'h_23A': 1.6342838983039245e-05,
 'v_23A': 0.0010038283680461335,
 'h_A': 4098.118273613894,
 'v_A': 13.156834942599902,
 'h_23B': 0.004329505087378093,
 'v_23B': 0.02263536968418067,
 'h_B': 1966.0597265924307,
 'v_B': 8.771223295066601,
 'h_31': 2.663338671985112e-05,
 'W_bomba': 11383.66202371387}

In [31]:
comercial_cost_dict = cost_function(comercial_diameters, return_cost_dict=True)
comercial_cost_dict


{'cost': 1410909.0619030888,
 'h_12': 111.1541886991869,
 'v_12': 2.4364509152962786,
 'h_23A': 55.88946801177149,
 'v_23A': 1.3705036398541564,
 'h_A': 4098.118273613894,
 'v_A': 13.156834942599902,
 'h_23B': 90444.70639537666,
 'v_23B': 35.084893180266405,
 'h_B': 1966.0597265924307,
 'v_B': 8.771223295066601,
 'h_31': 58.75812743620264,
 'W_bomba': 12010.889049336265}

In [32]:
np.abs((comercial_cost_dict["h_23A"] + comercial_cost_dict["h_A"]) -1*(comercial_cost_dict["h_23B"] + comercial_cost_dict["h_B"]))

88256.75838034342

In [26]:
ideal_cost_dict = cost_function(ideal_diameters, return_cost_dict=True)
comercial_cost_dict = cost_function(comercial_diameters, return_cost_dict=True)

In [27]:
h_12, v12 = aux.head_loss_12(i.Q, comercial_diameters[0], i.L12, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
h_23A, v23A = aux.head_loss_23A(i.Q, comercial_diameters[1], i.L23A, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)
h_23B, v23B = aux.head_loss_23B(i.Q, comercial_diameters[2], i.L23B, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)

h_31, v31 = aux.head_loss_31(i.Q, comercial_diameters[3], i.L31, i.Leqcot, i.Leqvalv, i.ρ, i.µ, i.ecom, return_velocity=True)

In [28]:
print("VA = ", v23A)
print("V31 = ", v31)

VA =  1.461870549177767
V31 =  1.3705036398541564


In [29]:
Q_A = v23A*(np.pi*(comercial_diameters[1]**2))/4
Q_B = v23B*(np.pi*(comercial_diameters[2]**2))/4

Q_12 = v12*(np.pi*(comercial_diameters[0]**2))/4



In [30]:
print(Q_A, Q_12, Q_B)

0.0016666666666666668 0.0027777777777777783 0.0011111111111111111
